# 小数据brugada(一维)

In [21]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.model_selection import train_test_split
# Enable inline plotting
data="D:/ecg/brugada2/brugada/"
data2="D:/ecg/brugada2/右束支阻滞/"
data3="D:/ecg/brugada2/正常/"
filelist = os.listdir(data)
print(filelist)
filelist1=os.listdir(data2)
print(filelist1)
filelist2=os.listdir(data3)
print(filelist2)
x_data=[]
for i in filelist:
    df=np.genfromtxt(data+i,delimiter=',')
    x_data.append(df)
y1=np.full((len(filelist),1),0)
for i in filelist1:
    df1=np.genfromtxt(data2+i,delimiter=',')
    x_data.append(df1)
y2=np.full((len(filelist1),1),1)
for i in filelist2:
    df2=np.genfromtxt(data3+i,delimiter=',')
    x_data.append(df2)   
y_data=np.concatenate((np.full((len(filelist),1),0),np.full((len(filelist1),1),1),(np.full((len(filelist2),1),2))),axis=0)
x_data=np.array([i for i in x_data])
print(x_data.shape)

['p578240-1.csv', 'p578240-10.csv', 'p578240-2.csv', 'p578240-3.csv', 'p578240-4.csv', 'p578240-5.csv', 'p578240-6.csv', 'p578240-7.csv', 'p578240-8.csv', 'p578240-9.csv', 'p663312-1.csv', 'p663312-10.csv', 'p663312-2.csv', 'p663312-3.csv', 'p663312-4.csv', 'p663312-5.csv', 'p663312-6.csv', 'p663312-7.csv', 'p663312-8.csv', 'p663312-9.csv', 'p769439-1.csv', 'p769439-10.csv', 'p769439-2.csv', 'p769439-3.csv', 'p769439-4.csv', 'p769439-5.csv', 'p769439-6.csv', 'p769439-7.csv', 'p769439-8.csv', 'p769439-9.csv', 'p825958-1.csv', 'p825958-10.csv', 'p825958-2.csv', 'p825958-3.csv', 'p825958-4.csv', 'p825958-5.csv', 'p825958-6.csv', 'p825958-7.csv', 'p825958-8.csv', 'p825958-9.csv', 'p848694-1.csv', 'p848694-10.csv', 'p848694-2.csv', 'p848694-3.csv', 'p848694-4.csv', 'p848694-5.csv', 'p848694-6.csv', 'p848694-7.csv', 'p848694-8.csv', 'p848694-9.csv', 'p849710-1.csv', 'p849710-10.csv', 'p849710-2.csv', 'p849710-3.csv', 'p849710-4.csv', 'p849710-5.csv', 'p849710-6.csv', 'p849710-7.csv', 'p84971

(490, 12, 216)


## FileDeal.py

In [26]:
import numpy as np
import os
def FileDeal(self,file1,file2,file3):
    #取文件下的目录
    filelist =os.listdir(file1)
    filelist1=os.listdir(file2)
    filelist2=os.listdir(file3)
    x=np.empty(shape=(0,216))
    for s,i in enumerate(filelist):#取数据
        df=np.genfromtxt(file1+i,delimiter=',')
        x=np.vstack((x,df))
        y1=np.full((s+1,1),0)
    for s,i in enumerate(filelist1):
        df1=np.genfromtxt(file2+i,delimiter=',')
        x=np.vstack((x,df1))
        y2=np.full((s+1,1),1)
    for s,i in enumerate(filelist2):
        df2=np.genfromtxt(file3+i,delimiter=',')
        x=np.vstack((x,df2))
        y3=np.full((s+1,1),2)
    y=np.concatenate((y1,y2,y3),axis=0)
    wholedata=np.hstack((y,x_data))
    return wholedata

## dataset.py

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset

#建立一个自己的数据集，创建一个将数据处理成DataLoader的类
class SignalDataset(Dataset):
    def __init__(self, raw_data):
        self._signal = torch.FloatTensor(raw_data[:,1:])#信号数据
        #self._fea_plus = torch.FloatTensor(raw_data[:, 1:3])
        #fea_plus由R前间隔和RR后间隔组成，后面将两个 RR 特征与注意力模块中提取的抽象特征连接起来
        self._label = torch.LongTensor(raw_data[:,0])#标签
        # self._emd = torch.FloatTensor(emd_data)


    @property
    def n_insts(self):
        ''' Property for dataset size '''
        '''数据集大小的属性'''
        return len(self._label)

    @property
    def sig_len(self):
        return self._signal.shape[1]

    def __len__(self):
        return self.n_insts

    def __getitem__(self, idx):
        return self._signal[idx],  self._label[idx]


## sublayer.py

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from config import SIG_LEN
#SIG_LEN = 241  
class MultiHeadAttention(nn.Module):#多头注意力
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):#n_head=4，d_model=64
        super().__init__()

        self.n_head = n_head#头的个数为3
        self.d_k = d_k
        self.d_v = d_v
        #三个参数矩阵
        self.w_qs = nn.Linear(d_model, n_head * d_k)
        self.w_ks = nn.Linear(d_model, n_head * d_k)
        self.w_vs = nn.Linear(d_model, n_head * d_v)
        nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_ks.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_vs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_v)))

        self.attention = SDPAttention(temperature=np.power(d_k, 0.5))
        self.layer_norm = nn.LayerNorm(d_model)

        self.fc = nn.Linear(n_head * d_v, d_model)
        nn.init.xavier_normal_(self.fc.weight)

        self.dropout = nn.Dropout(dropout)

    def forward(self, q, k, v, mask=None):
        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head

        sz_b, len_q, _ = q.size()
        sz_b, len_k, _ = k.size()
        sz_b, len_v, _ = v.size()

        residual = q

        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)

        q = q.permute(2, 0, 1, 3).contiguous().view(-1, len_q, d_k)  # (n*b) x lq x dk
        k = k.permute(2, 0, 1, 3).contiguous().view(-1, len_k, d_k)  # (n*b) x lk x dk
        v = v.permute(2, 0, 1, 3).contiguous().view(-1, len_v, d_v)  # (n*b) x lv x dv

        mask = mask.repeat(n_head, 1, 1)  # (n*b) x .. x ..
        output, attn = self.attention(q, k, v, mask=mask)

        output = output.view(n_head, sz_b, len_q, d_v)
        output = output.permute(1, 2, 0, 3).contiguous().view(sz_b, len_q, -1)  # b x lq x (n*dv)

        output = self.dropout(self.fc(output))
        output = self.layer_norm(output + residual)

        return output, attn


class SDPAttention(nn.Module):
    ''' Scaled Dot-Product Attention '''

    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=2)#在某一个维度进行softmax
        self.BN = nn.BatchNorm1d(SIG_LEN)#sig_len信号长度为241

    def forward(self, q, k, v, mask=None):
        attn = torch.bmm(q, k.transpose(1, 2))
        attn = attn / self.temperature

        if mask is not None:
            attn = attn.masked_fill(mask, -np.inf)

        attn = self.BN(attn)#对注意力进行批量归一化
        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = torch.bmm(attn, v)#输出，value的加权和

        return output, attn


## config.py

In [ ]:
batch_size = 16*4
d_model = 64
num_layers = 3
num_heads = 4
class_num = 4
d_inner = 512
dropout = 0.0
warm_steps = 4000
fea_num = 7
epoch = 100
PAD = 0
KS = 3
Fea_PLUS = 2
SIG_LEN = 241

## optim.py

In [ ]:
'''A wrapper class for optimizer'''
'''用于优化器的包装器类 '''
import numpy as np


class ScheduledOptim():
    '''A simple wrapper class for learning rate scheduling'''
    '''一个简单的学习速率调度的包装类  '''

    def __init__(self, optimizer, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0
        self.init_lr = np.power(d_model, -0.5)

    def step_and_update_lr(self):
        "Step with the inner optimizer"
        self._update_learning_rate()
        self._optimizer.step()

    def zero_grad(self):
        "Zero out the gradients by the inner optimizer"
        '''通过内部优化器消除梯度'''
        self._optimizer.zero_grad()

    def _get_lr_scale(self):
        return np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''
        '''每一步的学习速率调度'''

        self.n_current_steps += 1
        lr = self.init_lr * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr


##  FocalLoss

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class FocalLoss(nn.Module):#Focal 焦点损失函数
    def __init__(self, class_num, alpha=None, gamma=2, average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.average = average

        if alpha is None:
            self.alpha = torch.ones(class_num, 1)
        else:
            if isinstance(alpha, torch.Tensor):
                self.alpha = alpha
            else:
                self.alpha = torch.Tensor(alpha)

    def forward(self, inputs, targets, device):
        N, C = inputs.size()
        P = F.softmax(inputs, dim=1)

        class_mask = inputs.new_zeros(N, C)
        ids = targets.view(-1, 1)
        class_mask.scatter_(1, ids, 1.)
        if inputs.is_cuda and not self.alpha.is_cuda:
            self.alpha = self.alpha.to(device)
        alpha = self.alpha[ids.view(-1)]

        probs = (P * class_mask).sum(1).view(-1, 1)

        log_p = probs.log()

        batch_loss = -alpha * torch.pow((1 - probs),self.gamma) * log_p

        if self.average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()
        return loss


if __name__ == "__main__":
    FL = FocalLoss(class_num=5, gamma=2)
    CE = nn.CrossEntropyLoss()
    N = 4
    C = 5
    inputs = torch.rand(N, C)
    targets = torch.LongTensor(N).random_(C)
    inputs_fl = torch.tensor(inputs, requires_grad=True)
    targets_fl = torch.tensor(targets)

    inputs_ce = torch.tensor(inputs, requires_grad=True)
    targets_ce = torch.tensor(targets)
    print('----inputs----')
    print(inputs)
    print('---target-----')
    print(targets)

    fl_loss = FL(inputs_fl, targets_fl, device = torch.device('cuda:0'))
    ce_loss = CE(inputs_ce, targets_ce)
    print('ce = {}, fl ={}'.format(ce_loss.item(), fl_loss.item()))
    fl_loss.backward()
    ce_loss.backward()
    #print(inputs_fl.grad.data)
    print(inputs_ce.grad.data)

## roc

In [ ]:
# -*- coding:utf-8 -*-

import sys
import matplotlib.pyplot as plt
import numpy as np
import math
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.preprocessing import OneHotEncoder


def plot_roc(all_labels, all_pred):#画出roc曲线
    enc = OneHotEncoder()
    all_labels = np.array(all_labels)
    all_labels = all_labels[:,np.newaxis]
    label_h =enc.fit_transform(all_labels)
    label_h = label_h.toarray()
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(4):
        fpr[i], tpr[i], _ = roc_curve(label_h[:, i], all_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])


    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(4)]))
    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(4):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])
    # Finally average it and compute AUC
    mean_tpr /= 4
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    fpr["micro"], tpr["micro"], _ = roc_curve(label_h.ravel(), all_pred.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    lw=2
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.3f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.3f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red'])
    c = ['N','S','V','F']
    for i, color in zip(range(4), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.3f})'
                 ''.format(c[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve')
    plt.legend(loc="lower right")
    plt.show()


## block

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sublayer import MultiHeadAttention
from config import PAD
#mlp
class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''
    '''two-feed-forward-layer模块'''

    def __init__(self, d_in, d_hid, dropout=0.1):#d_hid=d_model=64
        super().__init__()
        self.w_1 = nn.Conv1d(d_in, d_hid, 1)  # position-wise
        self.w_2 = nn.Conv1d(d_hid, d_in, 1)  # position-wise
        self.layer_norm = nn.LayerNorm(d_in)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        residual = x
        output = x.transpose(1, 2)
        output = self.w_2(F.relu(self.w_1(output)))
        output = output.transpose(1, 2)
        output = self.dropout(output)
        output = self.layer_norm(output + residual)
        return output


def get_sinusoid_encoding_table(n_position, d_hid, padding_idx=None):#位置编码d_hid=d_model=64
    ''' Sinusoid position encoding table '''
    '''正弦位置编码表'''

    def cal_angle(position, hid_idx):
        return position / np.power(10000, 2 * (hid_idx // 2) / d_hid)

    def get_posi_angle_vec(position):
        return [cal_angle(position, hid_j) for hid_j in range(d_hid)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(n_position)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1
    #sinusoid相当于pe

    if padding_idx is not None:
        # zero vector for padding dimension
        #填充维数为零的向量
        sinusoid_table[padding_idx] = 0.

    return torch.FloatTensor(sinusoid_table)#返回位置编码


def get_non_pad_mask(seq):
    assert seq.dim() == 2#如果seq的维度不等于2，就抛出错误，如果seq的维度等于2就继续执行
    return seq.ne(PAD).type(torch.float).unsqueeze(-1)#ne不等于PAD=0位置返回ture,相等的位置返回false


def get_subsequent_mask(seq):#mask后面的向量
    ''' For masking out the subsequent info. '''
    '''用来屏蔽后续信息。'''

    sz_b, len_s = seq.size()
    subsequent_mask = torch.triu(
        torch.ones((len_s, len_s), device=seq.device, dtype=torch.uint8), diagonal=1)#triu上三角矩阵
    subsequent_mask = subsequent_mask.unsqueeze(0).expand(sz_b, -1, -1)  # b x ls x ls

    return subsequent_mask


def get_attn_key_pad_mask(seq_k, seq_q):#将pad的位置告诉模型，不让它影响模型的训练
    ''' For masking out the padding part of key sequence. '''
    '''用于屏蔽密钥序列的填充部分'''

    # Expand to fit the shape of key query attention matrix.
    #展开拟合键查询注意矩阵的形状
    len_q = seq_q.size(1)
    padding_mask = seq_k.eq(PAD)
    padding_mask = padding_mask.unsqueeze(1).expand(-1, len_q, -1)  # b x lq x lk

    return padding_mask

class EncoderLayer(nn.Module):
    ''' Compose with two layers '''
    '''用两层合成'''
    

    def __init__(self, d_model, d_inner, n_head, d_k, d_v, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.slf_attn = MultiHeadAttention(
            n_head, d_model, d_k, d_v, dropout=dropout)#多头注意力
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout=dropout)#MLP网络

    def forward(self, enc_input, non_pad_mask=None, slf_attn_mask=None):
        enc_output, enc_slf_attn = self.slf_attn(
            enc_input, enc_input, enc_input, mask=slf_attn_mask)#进入多头注意力机制
        enc_output *= non_pad_mask#enc_output=enc_output*non_pad_mask

        enc_output = self.pos_ffn(enc_output)#进入MLP
        enc_output *= non_pad_mask

        return enc_output, enc_slf_attn#返回输出和注意力

## model

In [ ]:
import torch
import torch.nn as nn
from block import get_sinusoid_encoding_table, get_attn_key_pad_mask, get_non_pad_mask, \
    get_subsequent_mask, EncoderLayer
from config import PAD, KS, Fea_PLUS


class Encoder(nn.Module):
    ''' A encoder model with self attention mechanism. '''

    def __init__(
            self,
            d_feature,
            n_layers, n_head, d_k, d_v,
            d_model, d_inner, dropout=0.1):
        super().__init__()

        n_position = d_feature + 1#d_feature为signal的长度，241
        self.src_word_emb = nn.Conv1d(1, d_model, kernel_size=KS, padding=int((KS - 1) / 2))#KS = 3卷积核的大小为三，d_model=512

        self.position_enc = nn.Embedding.from_pretrained(
            get_sinusoid_encoding_table(n_position, d_model, padding_idx=0),
            freeze=True)

        self.layer_stack = nn.ModuleList([
            EncoderLayer(d_model, d_inner, n_head, d_k, d_v, dropout=dropout)
            for _ in range(n_layers)])#sublayer

    def forward(self, src_seq, src_pos):
        # -- Prepare masks
        non_pad_mask = get_non_pad_mask(src_seq)#未被pad的部分
        slf_attn_mask = get_attn_key_pad_mask(seq_k=src_seq, seq_q=src_seq)#将pad的位置告诉模型，不让它影响模型的训练
        # -- Forward
        enc_output = src_seq.unsqueeze(1)
        enc_output = self.src_word_emb(enc_output)
        enc_output = enc_output.transpose(1, 2)
        enc_output.add_(self.position_enc(src_pos))#加上位置编码

        for enc_layer in self.layer_stack:
            enc_output, enc_slf_attn = enc_layer(
                enc_output,
                non_pad_mask=non_pad_mask,
                slf_attn_mask=slf_attn_mask)
        return enc_output,


class Transformer(nn.Module):
    ''' A sequence to sequence model with attention mechanism. '''
    def __init__(
            self, device,
            d_feature, d_model=512, d_inner=2048,
            n_layers=6, n_head=8, d_k=64, d_v=64, dropout=0.1,
            class_num=5):

        super().__init__()

        self.encoder = Encoder(d_feature, n_layers, n_head, d_k, d_v, d_model, d_inner, dropout)
        self.device = device

        self.linear1_cov = nn.Conv1d(d_feature, 1, kernel_size=1)
        self.linear1_linear = nn.Linear(d_model, class_num)
        # try different linear style
        self.linear2_cov = nn.Conv1d(d_model, 1, kernel_size=1)
        self.linear2_linear = nn.Linear(d_feature, class_num)

    def forward(self, src_seq):
        b, l = src_seq.size()
        src_pos = torch.LongTensor(
            [list(range(1, l + 1)) for i in range(b)]
        )
        src_pos = src_pos.to(self.device)

        enc_output, *_ = self.encoder(src_seq, src_pos)#进入encoder
        dec_output = enc_output
        res = self.linear1_cov(dec_output)#卷积
        res = res.contiguous().view(res.size()[0], -1)
        res = self.linear1_linear(res)#全连接层
        return res

## main

In [ ]:
# -*- coding: utf-8 -*-
import time
import sklearn
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm
import numpy as np
import pandas as pd
from model import Transformer
from optim import ScheduledOptim
from dataset import SignalDataset
from config import *
from FocalLoss import FocalLoss
from entropy import *
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
from roc import plot_roc
from imblearn.over_sampling import SMOTE
import time
import os

FL = FocalLoss(class_num=4, gamma=1.5, average=False)#损失函数


def cal_loss(pred, label, device):#损失
    cnt_per_class = np.zeros(4)#一共有四个类别
    loss = FL(pred, label, device)

    loss = F.cross_entropy(pred, label, reduction='sum')#交叉熵
    pred = pred.max(1)[1]#pred.max(1)对维度1取最大值,pred.max(1)[1]返回最大值的位置
    n_correct = pred.eq(label).sum().item()#得到预测最大值位置与标签相同的位置，并求和
    cnt_per_class = [cnt_per_class[j] + pred.eq(j).sum().item() for j in range(class_num)]
    return loss, n_correct, cnt_per_class


def cal_statistic(cm):#评价指标
    total_pred = cm.sum(0)#对维度0进行求和
    total_true = cm.sum(1)#对维度1进行求和
    # total_true = np.array([17703,   491,  1357,   159])
    # special acc, abnormal inlcuded only
    acc_SP = sum([cm[i, i] for i in range(1, class_num)]) / total_pred[0:class_num-1].sum()#混淆矩阵对角线加起来为预测对的，求准确率
    pre_i = [cm[i, i] / total_pred[i] for i in range(class_num)]#精准率
    rec_i = [cm[i, i] / total_true[i] for i in range(class_num)]#召回率
    F1_i = [2 * pre_i[i] * rec_i[i] / (pre_i[i] + rec_i[i]) for i in range(class_num)]#F1分数

    pre_i = np.array(pre_i)
    rec_i = np.array(rec_i)
    F1_i = np.array(F1_i)
    #将空值的地方设为0
    pre_i[np.isnan(pre_i)] = 0
    rec_i[np.isnan(rec_i)] = 0
    F1_i[np.isnan(F1_i)] = 0

    return acc_SP, list(pre_i), list(rec_i), list(F1_i)


def train_epoch(train_loader, device, model, optimizer, total_num):#训练集训练
    all_labels = []
    all_res = []
    model.train()#训练
    total_loss = 0
    total_correct = 0
    cnt_per_class = np.zeros(class_num)#分类类别
     #valod_loader迭代器，mininterval个更新最小间隔0.5,decs=进度条前缀,leave=false结束时不保存进度条
    for batch in tqdm(train_loader, mininterval=0.5, desc='- (Training)  ', leave=False):#加上训练进度条，以便于观察
        sig, fea_plus, label, = map(lambda x: x.to(device), batch)#每一批的信号与fea_plus,label
        # forward
        optimizer.zero_grad()#将梯度清零
        pred = model(sig, fea_plus)
        all_labels.extend(label.cpu().numpy())
        all_res.extend(pred.max(1)[1].cpu().numpy())
        # backward
        loss, n_correct, cnt = cal_loss(pred, label, device)#损失函数
        loss.backward()#反向传播梯度
        # update
        optimizer.step_and_update_lr()#根据梯度优化

        total_loss += loss.item()
        total_correct += n_correct
        cnt_per_class += cnt
    # cm = confusion_matrix(all_labels, all_res)
    # print(cm)
    # acc_SP, pre_i, rec_i, F1_i = cal_statistic(cm)
    # print('acc_SP is : {acc_SP}'.format(acc_SP=acc_SP))
    # print('pre_i is : {pre_i}'.format(pre_i=pre_i))
    # print('rec_i is : {rec_i}'.format(rec_i=rec_i))
    # print('F1_i is : {F1_i}'.format(F1_i=F1_i))
    train_loss = total_loss / total_num
    train_acc = total_correct / total_num
    return train_loss, train_acc, cnt_per_class
'''
iterable: 迭代器
desc: 进度条前缀
total: 设置steps, 默认len(iterable)
leave: 结束时是否保留进度条,默认True
file: 设置在哪里打印进度，默认sys.stderr. 可用file.write(str) 和file.flush() 来写入文件。
ncols: 设置进度条的宽度。如果指定则会动态调整去适应该宽度，如果不指定则会适配terminal的最大宽度，可能造成换行。 后面的计数和统计不限宽，如果设为0，则只打印计数和统计信息不显示进度条。(一般设置为70左右比较合适)
mininterval: 最小更新间隔,默认0.1s
maxinteravl: 最大更新间隔,默认10s
miniters: 最小更新间隔,默认10个iteration
ascii: 进度条的编码方式
position: 进度条打印间隔行数，默认0。一般用于嵌套的进度条
'''
def eval_epoch(valid_loader, device, model, total_num):#验证集训练
    all_labels = []
    all_res = []
    model.eval()
    total_loss = 0
    total_correct = 0
    cnt_per_class = np.zeros(class_num)
    with torch.no_grad():
        #valid_loader迭代器，mininterval个更新最小间隔0.5,decs=进度条前缀,leave=false结束时不保存进度条
        for batch in tqdm(valid_loader, mininterval=0.5, desc='- (Validation)  ', leave=False):
            sig, fea_plus, label, = map(lambda x: x.to(device), batch)#一个batch的数据
            # forward
            pred = model(sig, fea_plus)  # emd.contiguous().view(len(label), fea_num, -1)#预测值
            all_labels.extend(label.cpu().numpy())#标签
            all_res.extend(pred.max(1)[1].cpu().numpy())#用来画混淆矩阵
            loss, n_correct, cnt = cal_loss(pred, label, device)#损失函数，返回损失和正确

            total_loss += loss.item()#所有batch的损失
            total_correct += n_correct
            cnt_per_class += cnt
    cm = confusion_matrix(all_labels, all_res)#混淆矩阵
    print(cm)
    acc_SP, pre_i, rec_i, F1_i = cal_statistic(cm)
    print('acc_SP is : {acc_SP}'.format(acc_SP=acc_SP))#准确率
    print('pre_i is : {pre_i}'.format(pre_i=pre_i))#精准率
    print('rec_i is : {rec_i}'.format(rec_i=rec_i))#召回率
    print('F1_i is : {F1_i}'.format(F1_i=F1_i))#F1分数
    valid_loss = total_loss / total_num
    valid_acc = total_correct / total_num
    return valid_loss, valid_acc, cnt_per_class, sum(rec_i[1:]) * 0.6 + sum(pre_i[1:]) * 0.4


def test_epoch(valid_loader, device, model, total_num):#测试集训练
    all_labels = []
    all_res = []
    all_pres = []
    all_recs = []
    all_pred = []
    model.eval()
    total_loss = 0
    total_correct = 0
    cnt_per_class = np.zeros(class_num)
    with torch.no_grad():
        for batch in tqdm(valid_loader, mininterval=0.5, desc='- (Validation)  ', leave=False):
            sig, fea_plus, label, = map(lambda x: x.to(device), batch)
            # forward
            pred = model(sig, fea_plus)  # emd.contiguous().view(len(label), fea_num, -1)
            #pred是一维数据，label也是一维数据
            all_labels.extend(label.cpu().numpy())
            all_res.extend(pred.max(1)[1].cpu().numpy())
            all_pred.extend(pred.cpu().numpy())
            loss, n_correct, cnt = cal_loss(pred, label, device)

            total_loss += loss.item()
            total_correct += n_correct
            cnt_per_class += cnt

    # np.savetxt('all_pres.txt',all_pres)
    # np.savetxt('all_recs.txt', all_recs)
    np.savetxt('all_pred.txt',all_pred)#保存
    np.savetxt('all_label.txt', all_labels)
    all_pred = np.array(all_pred)
    plot_roc(all_labels,all_pred)
    cm = confusion_matrix(all_labels, all_res)
    print(cm)
    acc_SP, pre_i, rec_i, F1_i = cal_statistic(cm)
    print('acc_SP is : {acc_SP}'.format(acc_SP=acc_SP))
    print('pre_i is : {pre_i}'.format(pre_i=pre_i))
    print('rec_i is : {rec_i}'.format(rec_i=rec_i))
    print('F1_i is : {F1_i}'.format(F1_i=F1_i))
    test_acc = total_correct / total_num
    print('test_acc is : {test_acc}'.format(test_acc=test_acc))


if __name__ == '__main__':

    model_name = 'transform.chkpt'#加载模型
    file1="D:/ecg/brugada2/brugada/"
    file2="D:/ecg/brugada2/右束支阻滞/"
    file3="D:/ecg/brugada2/正常/"
    whole_data = FileDeal(file1,file2,file3)

    # a, b = SMOTE().fit_sample(whole_data[:, 1:], whole_data[:, 0])
    # whole_data = np.c_[b, a]

    for r in range(10):
        time_start_i = time.time()
        raw_train, raw_valid, _, _ = train_test_split(whole_data, list(whole_data[:, 0]), test_size=0.3,
                                                          random_state=r,stratify=list(whole_data[:, 0]))
        raw_valid, raw_test, _, _ = train_test_split(raw_valid, list(raw_valid[:, 0]), test_size=0.6666,
                                                     random_state=r,stratify=list(raw_valid[:, 0]))

        if torch.cuda.is_available():#如果有gpu
            device = torch.device('cuda:3')#运行设备是gpu
        else:
            device = torch.device('cpu')#运行设备是cgu
            
        
        #将数据变成自己的数据集以便于DataLoader
        train_data = SignalDataset(raw_train)
        valid_data = SignalDataset(raw_valid)
      
        train_loader = DataLoader(dataset=train_data,
                                  batch_size=batch_size,
                                  num_workers=2,
                                  shuffle=True)#训练集
        valid_loader = DataLoader(dataset=valid_data,
                                  batch_size=batch_size,
                                  num_workers=2,
                                  shuffle=True)#验证集
        test_loader = DataLoader(dataset=test_data,
                                 batch_size=batch_size,
                                 num_workers=2,
                                 shuffle=True)#测试集

        model = Transformer(device=device, d_feature=train_data.sig_len, d_model=d_model, d_inner=d_inner,
                            n_layers=num_layers, n_head=num_heads, d_k=64, d_v=64, dropout=dropout, class_num=class_num)#模型定义

        model = model.to(device)

        optimizer = ScheduledOptim(
            Adam(filter(lambda x: x.requires_grad, model.parameters()),
                 betas=(0.9, 0.98), eps=1e-09), d_model, warm_steps)#优化
        train_accs = []
        valid_accs = []
        eva_indis = []
        train_losses = []
        valid_losses = []
        for epoch_i in range(epoch):#训练次数
            print('[ Epoch', epoch_i, ']')
            start = time.time()
            #训练集效果
            train_loss, train_acc, cnt = train_epoch(train_loader, device, model, optimizer, train_data.__len__())#开始训练
            print('  - (Training)  loss: {loss: 8.5f}, accuracy: {accu:3.3f} %, '
                  'elapse: {elapse:3.3f} min'.format(loss=train_loss, accu=100 * train_acc,
                                                     elapse=(time.time() - start) / 60))
            print(cnt)
            train_accs.append(train_acc)
            train_losses.append(train_loss)
            start = time.time()
            #验证集效果
            valid_loss, valid_acc, cnt, eva_indi = eval_epoch(valid_loader, device, model, valid_data.__len__())
            print('  - (Validation)  loss: {loss: 8.5f}, accuracy: {accu:3.3f} %, '
                  'elapse: {elapse:3.3f} min'.format(loss=valid_loss, accu=100 * valid_acc,
                                                     elapse=(time.time() - start) / 60))
            print(cnt)
            valid_accs.append(valid_acc)
            eva_indis.append(eva_indi)
            valid_losses.append(valid_loss)
            model_state_dict = model.state_dict()

            checkpoint = {
                'model': model_state_dict,
                'config_file': 'config',
                'epoch': epoch_i}#模型保存

            if eva_indi >= max(eva_indis):
                torch.save(checkpoint, str(r)+model_name)
                print('    - [Info] The checkpoint file has been updated.')

        print('——ALL DONE!——')
        time_consume = (time.time() - time_start_i)
        print('total ' + str(time_consume) + 'seconds')
        plt.plot(valid_losses)
        plt.xlabel('epoch')
        plt.ylim([0.0, 0.5])
        plt.ylabel('valid loss')
        plt.title('loss change curve')
        plt.show()
        # pre = time.strft  ime("%Y-%m-%d_%H:%M:%S", time.localtime())
        # config = [batch_size, d_model, num_layers, num_heads, class_num, d_inner, dropout]
        # np.save(pre + '_config.npy', config)
        # np.save(pre + '_train_accs.npy', train_accs)
        # np.save(pre + '_valid_accs.npy', valid_accs)
        # np.save(pre + '_train_losses.npy', train_losses)
        # np.save(pre + '_valid_losses.npy', valid_losses)
        #测试集效果
        test_model_name = str(r) + model_name
        model = Transformer(device=device, d_feature=test_data.sig_len, d_model=d_model, d_inner=d_inner,
                            n_layers=num_layers, n_head=num_heads, d_k=64, d_v=64, dropout=dropout,
                            class_num=class_num)
        chkpoint = torch.load(test_model_name, map_location='cuda:3')
        model.load_state_dict(chkpoint['model'])
        model = model.to(device)
        test_epoch(test_loader, device, model, test_data.__len__())

    # models = []
    # for r in range(10):
    #     test_model_name = str(r) + model_name
    #     model = Transformer(device=device, d_feature=test_data.sig_len, d_model=d_model, d_inner=d_inner,
    #                         n_layers=num_layers, n_head=num_heads, d_k=64, d_v=64, dropout=dropout,
    #                         class_num=class_num)
    #     chkpoint = torch.load(test_model_name)
    #     model.load_state_dict(chkpoint['model'])
    #     model = model.to(device)
    #     models.append(model)
    # voting_epoch(test_loader, device, models, test_data.__len__())
